In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.dates as dates

df_weather = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/63aee981839ba6becfdedfc5fca77a152c08ebc38e3dd05f7c85e14d.csv')

# Remove leap days
df_weather = df_weather[df_weather.Date.str[5:] != '02-29']
# To used as index for adding 2015 dataSerie
idx_day = np.arange(1,366)

def getDataFrame (start, end, element, col_name):
    df = df_weather.loc[(df_weather['Element'] == element )]
    df_return = df.loc[(df['Date'] >= start)  & (df['Date'] <= end)]
    if element == 'TMIN':
        df_return = df_return.loc[df_return.groupby(df_return['Date'].str[5:])['Data_Value'].idxmin()]
    else:
        df_return = df_return.loc[df_return.groupby(df_return['Date'].str[5:])['Data_Value'].idxmax()]
    df_return.drop(['ID', 'Date', 'Element'], axis=1, inplace=True)
    df_return.Data_Value = df_return.Data_Value/10 
    df_return.columns = [col_name]
    df_return.index = idx_day
    return df_return


# Min temperatures
df_min = getDataFrame('2005-01-01', '2014-12-31', 'TMIN', 'min_05_14')
df_min_2015 = getDataFrame('2015-01-01', '2015-12-31', 'TMIN', 'min_2015')
df_min['min_2015'] = df_min_2015['min_2015']
# Set 2015 values higher than min record in period 2015-2014 to NaN since not needed 
for idx, row in df_min.iterrows():
        if row[0] <= row[1]:
            row[1] = np.nan

            
# Max temperatures
df_max = getDataFrame('2005-01-01', '2014-12-31', 'TMAX', 'max_05_14')
df_max_2015 = getDataFrame('2015-01-01', '2015-12-31', 'TMAX', 'max_2015')
df_max['max_2015'] = df_max_2015['max_2015']
# Set 2015 values lower than max record in period 2015-2014 to NaN since not needed
for idx, row in df_max.iterrows():
        if row[0] >= row[1]:
            row[1] = np.nan
        

######################
# Plot configuration #
######################

fig = plt.figure()

observation_dates = np.arange('2015-01-01', '2016-01-01', dtype='datetime64[D]')
observation_dates = list(map(pd.to_datetime, observation_dates))

lows = df_min.min_05_14.values
highs = df_max.max_05_14.values

plt.plot(observation_dates, lows, '-', color='orange', linewidth=1, label="Min T recorded in 2005-2014")
plt.plot(observation_dates, highs, '-', color='blue', linewidth=1, label="Max T recorded in 2005-2014")
plt.fill_between(observation_dates, lows, highs, facecolor='blue', alpha=.25)

plt.scatter(observation_dates, df_min.min_2015, color='blue', marker="v", label="2015 Min T lower than record")
plt.scatter(observation_dates, df_max.max_2015, color='orange', marker="^", label="2015 Max T higher than record")

plt.legend(loc="best", frameon=False, prop={'size':8})
plt.title('Record low and high temperatures, Norway', size=10)
plt.ylabel('Record temperature $^\circ$C')

#Get current axes
ax = plt.gca()
# Hide the right and top spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.xaxis.set_major_locator(dates.MonthLocator(bymonthday=1))
ax.xaxis.set_major_formatter(dates.DateFormatter('%b'))

plt.yticks(fontsize=8, color='black')
plt.xticks(fontsize=8, color='black')

ax.set_xlim(pd.to_datetime(observation_dates[0]), pd.to_datetime(observation_dates[-1]))
fig.autofmt_xdate()